In [2]:
# Dependencies
import pandas as pd
import numpy as np
import os


import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [3]:
# CSV files
metadata = pd.read_csv("DataSets/Belly_Button_Biodiversity_Metadata.csv")
otu_id = pd.read_csv("DataSets/belly_button_biodiversity_otu_id.csv")
samples = pd.read_csv("DataSets/belly_button_biodiversity_samples.csv", dtype =object)

In [4]:
samples.set_index(["otu_id"],drop=True,inplace=True)
samples.head()

,BB_940,BB_941,BB_943,BB_944,BB_945,BB_946,BB_947,BB_948,BB_949,BB_950,...,BB_1562,BB_1563,BB_1564,BB_1572,BB_1573,BB_1574,BB_1576,BB_1577,BB_1581,BB_1601
otu_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
samples = pd.DataFrame(samples)
samples = samples.T
samples.head()


otu_id,1,2,3,4,5,6,7,8,9,10,...,3665,3666,3667,3668,3669,3670,3671,3672,3673,3674
BB_940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BB_941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BB_943,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BB_944,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BB_945,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
samples.index.names=["sample"]

In [7]:
metadata.head()

,SAMPLEID,EVENT,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION,COUNTRY012,ZIP012,...,DOG,CAT,IMPSURFACE013,NPP013,MMAXTEMP013,PFC013,IMPSURFACE1319,NPP1319,MMAXTEMP1319,PFC1319
0,940,BellyButtonsScienceOnline,Caucasian,F,24.0,2.0,I,Beaufort/NC,usa,22306,...,no,no,8852.0,37.172222,54.5,NaN,1,NaN,33.990002,25.5
1,941,NaN,Caucasian/Midleastern,F,34.0,1.0,I,Chicago/IL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,943,BellyButtonsScienceOnline,Caucasian,F,49.0,1.0,I,Omaha/NE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,944,BellyButtonsScienceOnline,European,M,44.0,1.0,I,NewHaven/CT,usa,7079,...,no,yes,NaN,35.816666,16.0,NaN,0,6567.0,32.403332,28.5
4,945,BellyButtonsScienceOnline,Caucasian,F,48.0,1.0,I,Philidelphia/PA,usa,84404,...,no,no,NaN,37.783333,4.0,NaN,0,5613.0,33.634445,24.0


In [8]:
metadata.columns

Index(['SAMPLEID', 'EVENT', 'ETHNICITY', 'GENDER', 'AGE', 'WFREQ', 'BBTYPE',
       'LOCATION', 'COUNTRY012', 'ZIP012', 'COUNTRY1319', 'ZIP1319', 'DOG',
       'CAT', 'IMPSURFACE013', 'NPP013', 'MMAXTEMP013', 'PFC013',
       'IMPSURFACE1319', 'NPP1319', 'MMAXTEMP1319', 'PFC1319'],
      dtype='object')

In [9]:
metadata = metadata[['SAMPLEID', 'ETHNICITY', 'GENDER', 'AGE', 'WFREQ', 'BBTYPE',
       'LOCATION']]
metadata.SAMPLEID = metadata.SAMPLEID.astype(str)
metadata.dtypes

SAMPLEID      object
ETHNICITY     object
GENDER        object
AGE          float64
WFREQ        float64
BBTYPE        object
LOCATION      object
dtype: object

In [10]:
metadata.head()

,SAMPLEID,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION
0,940,Caucasian,F,24.0,2.0,I,Beaufort/NC
1,941,Caucasian/Midleastern,F,34.0,1.0,I,Chicago/IL
2,943,Caucasian,F,49.0,1.0,I,Omaha/NE
3,944,European,M,44.0,1.0,I,NewHaven/CT
4,945,Caucasian,F,48.0,1.0,I,Philidelphia/PA


In [11]:
metadata['SAMPLEID'] = "BB_" + metadata["SAMPLEID"]

In [12]:
metadata.to_csv("clean_metadata.csv",index=False)
otu_id.to_csv("clean_otu.csv",index=False)
samples.to_csv("clean_samples.csv",index=False)

In [13]:
engine = create_engine("sqlite:///belly_button_biodiversity.sqlite")
conn = engine.connect()

In [29]:
Base = declarative_base()

class OTU(Base):
    __tablename__ = 'otu'

    otu_id = Column(Integer, primary_key=True)
    lowest_xfound = Column(Text)
    
    def __repr__(self):
        return f"ID: {self.otu_id}, NAME: {self.lowest_taxonomic_unit_found}"
    
Base.metadata.create_all(engine)

In [15]:
data_to_insert = pd.read_csv("clean_otu.csv")
data = data_to_insert.to_dict(orient="records")
metadata = MetaData(bind=engine)
metadata.reflect()

In [16]:
table = sqlalchemy.Table('otu', metadata, autoload=True)


In [17]:
conn.execute(table.insert(), data)

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: otu.otu_id [SQL: 'INSERT INTO otu (otu_id, lowest_taxonomic_unit_found) VALUES (?, ?)'] [parameters: ((1, 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus'), (2, 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus'), (3, 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Natronorubrum'), (4, 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter'), (5, 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter'), (6, 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter'), (7, 'Bacteria'), (8, 'Bacteria')  ... displaying 10 of 3674 total bound parameter sets ...  (3673, 'Unclassified'), (3674, 'Unclassified'))]

In [18]:
conn.execute("select * from otu limit 5").fetchall()


[(1, 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus'),
 (2, 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus'),
 (3, 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Natronorubrum'),
 (4, 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter'),
 (5, 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter')]

In [24]:
engine = create_engine("sqlite:///belly_button_biodiversity.sqlite")
conn = engine.connect()

In [25]:
Base = declarative_base()

class Metadata(Base):
    __tablename__ = 'metadata'

    SAMPLEID = Column(Text, primary_key=True)
    ETHNICITY = Column(Text)
    GENDER = Column(Text)
    AGE = Column(Float)
    WFREQ = Column(Float)
    BBTYPE = Column(Text)
    LOCATION = Column(Text)
    
    def __repr__(self):
        return f"ID: {self.SAMPLEID}, {self.ETHNICITY} {self.GENDER}, (self.AGE), {self.LOCATION} type: {self.BBTYPE}, wfreq: {self.WFREQ}"
Base.metadata.create_all(engine)

In [26]:
data_to_insert = pd.read_csv("clean_metadata.csv")
data = data_to_insert.to_dict(orient="records")

In [27]:
data

[{'AGE': 24.0,
  'BBTYPE': 'I',
  'ETHNICITY': 'Caucasian',
  'GENDER': 'F',
  'LOCATION': 'Beaufort/NC',
  'SAMPLEID': 'BB_940',
  'WFREQ': 2.0},
 {'AGE': 34.0,
  'BBTYPE': 'I',
  'ETHNICITY': 'Caucasian/Midleastern',
  'GENDER': 'F',
  'LOCATION': 'Chicago/IL',
  'SAMPLEID': 'BB_941',
  'WFREQ': 1.0},
 {'AGE': 49.0,
  'BBTYPE': 'I',
  'ETHNICITY': 'Caucasian',
  'GENDER': 'F',
  'LOCATION': 'Omaha/NE',
  'SAMPLEID': 'BB_943',
  'WFREQ': 1.0},
 {'AGE': 44.0,
  'BBTYPE': 'I',
  'ETHNICITY': 'European',
  'GENDER': 'M',
  'LOCATION': 'NewHaven/CT',
  'SAMPLEID': 'BB_944',
  'WFREQ': 1.0},
 {'AGE': 48.0,
  'BBTYPE': 'I',
  'ETHNICITY': 'Caucasian',
  'GENDER': 'F',
  'LOCATION': 'Philidelphia/PA',
  'SAMPLEID': 'BB_945',
  'WFREQ': 1.0},
 {'AGE': 42.0,
  'BBTYPE': 'I',
  'ETHNICITY': 'Caucasian',
  'GENDER': 'F',
  'LOCATION': 'Deerfield/MA',
  'SAMPLEID': 'BB_946',
  'WFREQ': 3.0},
 {'AGE': 49.0,
  'BBTYPE': 'I',
  'ETHNICITY': 'Caucasian',
  'GENDER': 'M',
  'LOCATION': 'ChapelHill/NC'

In [28]:
metadata = MetaData(bind=engine)
metadata.reflect()
table = sqlalchemy.Table('metadata', metadata, autoload=True)
conn.execute(table.insert(), data)

In [ ]:
conn.execute("select * from metadata limit 5").fetchall()